<a href="https://colab.research.google.com/github/AurelienNioche/InteractiveInference/blob/aurelien/vae_predictor__collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

import time

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device", device)

Using device cuda


In [4]:
batch_size = 256
epochs = 10
fig_folder = "results"
backup_folder = "backup"

for f in fig_folder, backup_folder:
    os.makedirs(f, exist_ok=True)

In [5]:
train_loader = DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [6]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([256, 1, 28, 28])

In [7]:
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc1 = nn.Conv2d(1, 32, 3, padding="same")
        self.enc2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.enc3 = nn.Conv2d(64, 64, 3, padding="same")
        self.enc4 = nn.Conv2d(64, 64, 3, padding="same")
        self.enc5 = nn.Linear(12544, 32)

        self.dec1 = nn.Linear(2, 12544)
        self.dec2 = nn.ConvTranspose2d(64, 32, 3, stride=(2, 2), padding=1, output_padding=1)
        self.dec3 = nn.Conv2d(32, 1, 3, padding="same")

        self.mu = nn.Linear(32, 2)
        self.logvar = nn.Linear(32, 2)

    def encode(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc4(x))
        x = x.flatten(start_dim=1)
        x = F.relu(self.enc5(x))
        mu = self.mu(x)
        logvar = self.logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):

        z = F.relu(self.dec1(z))
        z = z.view(-1, 64, 14, 14)
        z = F.relu(self.dec2(z))
        x = torch.sigmoid(self.dec3(z))
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [8]:
# Reconstruction + KL divergence losses summed over all elements and batch
def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [9]:
def train(model, loss_function, epoch, log_interval=100):
    model.train()
    train_loss = 0
    t = time.time()
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \t {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data),
                time.time() - t))
            t = time.time()

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(model, loss_function, epoch, batch_size, fig_folder):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         f'{fig_folder}/reconstruction_{epoch}.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [11]:
vae_model = VAE().to(device)

optimizer = optim.Adam(vae_model.parameters(), lr=1e-3)


for epoch in range(1, epochs + 1):
    train(model=vae_model, loss_function=vae_loss, epoch=epoch)
    test(model=vae_model, loss_function=vae_loss, epoch=epoch, batch_size=batch_size,
         fig_folder=fig_folder)
    with torch.no_grad():
        sample = torch.randn(64, 2).to(device)
        sample = vae_model.decode(sample)
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 542.950745 	 0.07
Train Epoch: 1 [25600/60000 (43%)]	Loss: 223.091507 	 6.71
Train Epoch: 1 [51200/60000 (85%)]	Loss: 187.280441 	 5.42
====> Epoch: 1 Average loss: 241.5154
====> Test set loss: 183.5688
Train Epoch: 2 [0/60000 (0%)]	Loss: 180.575928 	 0.07
Train Epoch: 2 [25600/60000 (43%)]	Loss: 177.568848 	 5.51
Train Epoch: 2 [51200/60000 (85%)]	Loss: 173.076874 	 6.54
====> Epoch: 2 Average loss: 178.3417
====> Test set loss: 175.6604
Train Epoch: 3 [0/60000 (0%)]	Loss: 180.367218 	 0.06
Train Epoch: 3 [25600/60000 (43%)]	Loss: 170.229630 	 5.52
Train Epoch: 3 [51200/60000 (85%)]	Loss: 172.130875 	 6.58
====> Epoch: 3 Average loss: 171.6273
====> Test set loss: 168.8900
Train Epoch: 4 [0/60000 (0%)]	Loss: 167.728760 	 0.06
Train Epoch: 4 [25600/60000 (43%)]	Loss: 162.269592 	 6.08
Train Epoch: 4 [51200/60000 (85%)]	Loss: 161.889801 	 5.53
====> Epoch: 4 Average loss: 165.8841
====> Test set loss: 163.7053
Train Epoch: 5 [0/60000 (0%)]	Loss: 163.

In [ ]:
backup_file_vae = f"{backup_folder}/vae_model.p"
torch.save(vae_model.state_dict(), backup_file_vae)

In [ ]:
model = VAE()
model.load_state_dict(torch.load(backup_file_vae))